# 评价特征
+	重要性（ 出度中心性、介数中心性、紧密中心性）

这个参考14论文

+	所在领域竞争力（所在领域的艺术家人数，所在领域人数越多，影响力越大）

举例说明：同一水平的英文作家的影响力肯定比祖鲁语作家的影响力高，可以理解为作品类型本身的传播能力

+	生涯期间所在领域人数

举例说明：百家争鸣时代的普通天才往往能在星光黯淡的时代大放异彩，可以理解为艺术在所在领域在艺术家活跃期间获取影响力的难度

+	所在领域年份影响力排名（先筛选出艺术家所在领域此年份的所有人，然后根据影响人数进行排名）（排名越小，影响力越大）

很直观的指标

+	所在年份竞争力（这一年份的所有领域的艺术家总人数）（竞争力越大，影响力越大）

艺术家瓜分影响力这块蛋糕的难度（45用相关性分析后续再决定选择）

+	当前年代追随者数目

直观

+	（非当前年代追随者数目）

艺术家在创作生涯结束后对未来时代的影响力衡量

+	追随者跨的领域数目

艺术家作品对非本领域艺术家的吸引程度，也可以说是艺术家作品所含的纯粹的艺术性的度量

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import string
import pandas as pd 
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import networkx as nx  


In [417]:
data=pd.read_csv('./2021_ICM_Problem_D_Data/influence_data.csv',encoding='gbk')

In [3]:
data.head()

,influencer_id,influencer_name,influencer_main_genre,influencer_active_start,follower_id,follower_name,follower_main_genre,follower_active_start
0,759491,The Exploited,Pop/Rock,1980,74,Special Duties,Pop/Rock,1980
1,25462,Tricky,Electronic,1990,335,PJ Harvey,Pop/Rock,1990
2,66915,Bob Dylan,Pop/Rock,1960,335,PJ Harvey,Pop/Rock,1990
3,71209,Leonard Cohen,Pop/Rock,1950,335,PJ Harvey,Pop/Rock,1990
4,91438,The Gun Club,Pop/Rock,1980,335,PJ Harvey,Pop/Rock,1990


# 异常值检测和处理

In [148]:
data[data['influencer_active_start']>data['follower_active_start']].head()#不将这些influencer比follewer晚的作为异常值
#由此可以证明艺术家创作活跃时间有限

,influencer_id,influencer_name,influencer_main_genre,influencer_active_start,follower_id,follower_name,follower_main_genre,follower_active_start
190,145443,Fito Olivares,Latin,1990,4852,Selena,Latin,1980
320,752945,Paul van Dyk,Electronic,1990,6232,Robert Miles,Electronic,1980
358,172642,Joe Mooney,Vocal,1950,6334,Tony Bennett,Vocal,1940
362,212612,Johnny Hartman,Vocal,1950,6334,Tony Bennett,Vocal,1940
390,307135,Rick Nielsen,Pop/Rock,1970,7230,Brian May,Pop/Rock,1960


# 制作networkx 有向图对象

In [95]:
G=nx.DiGraph()

In [96]:
influencer=[]
follower=[]
for line in data.iterrows():
    influencer.append(line[1]['influencer_id'])
    follower.append(line[1]['follower_id'])


In [98]:
for i in range(len(influencer)):
    G.add_edge(influencer[i],follower[i])#此处有向图的指向是1指向2

In [99]:
nodes=G.node()

# 异常值处理2
+ id为[893383, 303506, 957340]的influencer出现了统计错误，分别把他们的一个follower作为id对应的influncer的名字和年份了

我们将这类错误行全部删除

In [19]:
error_list
#id=893383的全删除，因为两只乐队都不多且混杂严重
#删除id=303506，姓名为Day26的一行（因为其它24行是usher）

[893383, 303506, 957340]

In [27]:
data[data['influencer_id']==893383]

,influencer_id,influencer_name,influencer_main_genre,influencer_active_start,follower_id,follower_name,follower_main_genre,follower_active_start
6450,893383,The Muffs,Pop/Rock,1990,97200,Bracket,Pop/Rock,1990
6644,893383,The Kills,Pop/Rock,1990,99508,The Kills,Pop/Rock,2000
18669,893383,The Muffs,Pop/Rock,1990,329232,New Found Glory,Pop/Rock,1990
20046,893383,The Muffs,Pop/Rock,1990,366681,Paramore,Pop/Rock,2000
26355,893383,The Muffs,Pop/Rock,1990,585574,Sum 41,Pop/Rock,1990
33376,893383,The Muffs,Pop/Rock,1990,784535,The Donnas,Pop/Rock,1990
41095,893383,The Muffs,Pop/Rock,1990,1013109,Hey Monday,Pop/Rock,2000


In [34]:
data.drop([6644,29381,37697],inplace=True)

# factor1 重要度计算

In [35]:
deg=nx.out_degree_centrality(G)#出度中心性

In [36]:
bet=nx.betweenness_centrality(G)#介数中心性

In [37]:
clo=nx.closeness_centrality(G)#紧密度中心性

In [48]:
maxclo_key=0
maxclo_value=0
for i in clo:
    if clo[i]>maxclo_value:
        maxclo_value=clo[i]
        maxclo_key=i

In [49]:
maxbet_key=0
maxbet_value=0
for i in bet:
    if bet[i]>maxbet_value:
        maxbet_value=bet[i]
        maxbet_key=i

In [50]:
maxdeg_key=0
maxdeg_value=0
for i in deg:
    if deg[i]>maxdeg_value:
        maxdeg_value=bet[i]
        maxdeg_key=i

In [56]:
#每个人的向量
arrays={}
for node in nodes:
    array=[]
    array.append(deg[node]/maxdeg_value)
    array.append(bet[node]/maxbet_value)
    array.append(clo[node]/maxclo_value)
    arrays[node]=array

In [57]:
idea_array=[1,1,0]

In [58]:
#算距离
distances={}
for array in arrays:
    distance=np.square(arrays[array][0]-idea_array[0])+np.square(arrays[array][1]-idea_array[1])+np.square(arrays[array][2]-idea_array[2])
    distance=np.sqrt(distance)
    distances[array]=distance

# factor2 所在领域竞争力（所在领域的艺术家人数，所在领域人数越多，影响力越大）

In [106]:
influencer_id=data['influencer_id'].unique()

In [84]:
valuecount=data['influencer_main_genre'].value_counts()

In [460]:
#有部分只属于follwer不属于influencer
field={}#各个艺术家所处的领域
for influencer in influencer_id:
    field[influencer]=data[data['influencer_id']==influencer]['influencer_main_genre'].values[0]



In [108]:
field_num={}#各个influencer对应的竞争力
for influencer in influencer_id:
    field_num[influencer]=valuecount[field[influencer]] 

In [113]:
follower_id=data['follower_id'].unique()

In [114]:
follower_id=[i for i in follower_id if i not in influencer_id]#提取只存在于follower的艺术家列表

In [123]:
valuecount_2=data['follower_main_genre'].value_counts()

In [464]:
for follower in follower_id:
    field[follower]=data[data['follower_id']==follower]['follower_main_genre'].values[0]

In [124]:
for follower in follower_id:
    field_num[follower]=valuecount_2[field[follower]] 

In [135]:
df=pd.DataFrame([field]).T
type_count=df[0].value_counts()

In [140]:
field_num={}#各个influencer对应的竞争力
for influencer in influencer_id:
    field_num[influencer]=type_count[field[influencer]]
for follower in follower_id:
    field_num[follower]=type_count[field[follower]]

In [465]:
pd.DataFrame([field_num]).T[0].value_counts()#核对无误

2808    2808
677      677
406      406
403      403
229      229
208      208
162      162
141      141
101      101
95        95
89        89
81        81
50        50
46        46
38        38
28        28
23        23
11        11
4          4
3          3
Name: 0, dtype: int64

In [466]:
pd.DataFrame([field]).T[0].value_counts()

Pop/Rock          2808
R&B;               677
Jazz               406
Country            403
Latin              229
Electronic         208
Vocal              162
Reggae             141
Blues              101
Folk                95
Religious           89
International       81
Stage & Screen      50
Comedy/Spoken       46
New Age             38
Classical           28
Easy Listening      23
Avant-Garde         11
Children's           4
Unknown              3
Name: 0, dtype: int64

# factor3 生涯期间所在领域人数
统计年份是1930~2010，考虑年份是1930~2040

超过30年则认为是不同时代例如（1970最少与2000不同时代）

例如有一个1970开始活跃的R&B艺术家，那么他会使得1970、1980、1990的R&B领域活跃人数+1，然后算70、80、90三个年代的R&B领域活跃人数均值作为他的所在领域年份人数

In [152]:
data.head()

,influencer_id,influencer_name,influencer_main_genre,influencer_active_start,follower_id,follower_name,follower_main_genre,follower_active_start
0,759491,The Exploited,Pop/Rock,1980,74,Special Duties,Pop/Rock,1980
1,25462,Tricky,Electronic,1990,335,PJ Harvey,Pop/Rock,1990
2,66915,Bob Dylan,Pop/Rock,1960,335,PJ Harvey,Pop/Rock,1990
3,71209,Leonard Cohen,Pop/Rock,1950,335,PJ Harvey,Pop/Rock,1990
4,91438,The Gun Club,Pop/Rock,1980,335,PJ Harvey,Pop/Rock,1990


In [153]:
age_sati={1930:{},1940:{},1950:{},1960:{},1970:{},1980:{},1990:{},2000:{},2010:{},2020:{},2030:{}}

In [161]:
a=data['influencer_main_genre'].unique()
b=data['follower_main_genre'].unique()
type_sum=a#所有领域之和
for new_type in b:
    if new_type not in type_sum:
        type_sum.append(new_type)

In [164]:
for a_type in type_sum:
    for j in age_sati:
        age_sati[j][a_type]=0#统计字典初始化

In [166]:
for influencer in influencer_id:
    age_id=data[data['influencer_id']==influencer]['influencer_active_start'].values[0]
    type_id=data[data['influencer_id']==influencer]['influencer_main_genre'].values[0]
    age_sati[age_id][type_id]=age_sati[age_id][type_id]+1
    age_sati[age_id+10][type_id]=age_sati[age_id][type_id]+1
    age_sati[age_id+20][type_id]=age_sati[age_id][type_id]+1

In [173]:
for follower in follower_id:
    age_id=data[data['follower_id']==follower]['follower_active_start'].values[0]
    type_id=data[data['follower_id']==follower]['follower_main_genre'].values[0]
    age_sati[age_id][type_id]=age_sati[age_id][type_id]+1
    age_sati[age_id+10][type_id]=age_sati[age_id][type_id]+1
    age_sati[age_id+20][type_id]=age_sati[age_id][type_id]+1

In [176]:
field_age_people={}#生涯期间所在领域人数
for influencer in influencer_id:
    age_id=data[data['influencer_id']==influencer]['influencer_active_start'].values[0]
    type_id=data[data['influencer_id']==influencer]['influencer_main_genre'].values[0]
    field_age_people[influencer]=age_sati[age_id][type_id]+age_sati[age_id+10][type_id]+age_sati[age_id+20][type_id]

In [177]:
for follower in follower_id:
    age_id=data[data['follower_id']==follower]['follower_active_start'].values[0]
    type_id=data[data['follower_id']==follower]['follower_main_genre'].values[0]
    field_age_people[follower]=age_sati[age_id][type_id]+age_sati[age_id+10][type_id]+age_sati[age_id+20][type_id]

# factor4 所在领域年份影响力排名
1970、1980、1990三年所在领域的影响力排名均值

In [271]:
age_sati_4={1930:{},1940:{},1950:{},1960:{},1970:{},1980:{},1990:{},2000:{},2010:{},2020:{},2030:{}}
a=data['influencer_main_genre'].unique()
b=data['follower_main_genre'].unique()
type_sum=a#所有领域之和
for new_type in b:
    if new_type not in type_sum:
        type_sum.append(new_type)
for a_type in type_sum:
    for j in age_sati_4:
        age_sati_4[j][a_type]={}#统计字典初始化

In [272]:
for influencer in influencer_id:
    age_id=data[data['influencer_id']==influencer]['influencer_active_start'].values[0]
    type_id=data[data['influencer_id']==influencer]['influencer_main_genre'].values[0]
    age_sati_4[age_id][type_id][influencer]=distances[influencer]
    age_sati_4[age_id+10][type_id][influencer]=distances[influencer]
    age_sati_4[age_id+20][type_id][influencer]=distances[influencer]

In [273]:
for follower in follower_id:
    age_id=data[data['follower_id']==follower]['follower_active_start'].values[0]
    type_id=data[data['follower_id']==follower]['follower_main_genre'].values[0]
    age_sati_4[age_id][type_id][follower]=distances[follower]
    age_sati_4[age_id+10][type_id][follower]=distances[follower]
    age_sati_4[age_id+20][type_id][follower]=distances[follower]

In [270]:
import copy

In [276]:
age_sati_rank=copy.deepcopy(age_sati_4)#.-年份-领域-id-对应影响力排名（1~max）
for i in age_sati_rank:
    for j in age_sati_rank[i]:
        un_dict=age_sati_rank[i][j]
        sorted_re=sorted(un_dict.items(), key = lambda kv:(kv[1], kv[0]))[::-1]
        for l in range(len(sorted_re)):
            age_sati_rank[i][j][sorted_re[l][0]]=l+1

In [364]:
data.head()

,influencer_id,influencer_name,influencer_main_genre,influencer_active_start,follower_id,follower_name,follower_main_genre,follower_active_start
0,759491,The Exploited,Pop/Rock,1980,74,Special Duties,Pop/Rock,1980
1,25462,Tricky,Electronic,1990,335,PJ Harvey,Pop/Rock,1990
2,66915,Bob Dylan,Pop/Rock,1960,335,PJ Harvey,Pop/Rock,1990
3,71209,Leonard Cohen,Pop/Rock,1950,335,PJ Harvey,Pop/Rock,1990
4,91438,The Gun Club,Pop/Rock,1980,335,PJ Harvey,Pop/Rock,1990


In [394]:
age_sati_rank_new={}#所在领域生涯影响力排名
for follower in follower_id:
    age=data[data['follower_id']==follower]['follower_active_start'].values[0]
    field=data[data['follower_id']==follower]['follower_main_genre'].values[0]
    age_sati_rank_new[follower]=(age_sati_rank[age][field][follower]+age_sati_rank[age+10][field][follower]+age_sati_rank[age+20][field][follower])/3

In [395]:
for influencer in influencer_id:
    age=data[data['influencer_id']==influencer]['influencer_active_start'].values[0]
    field=data[data['influencer_id']==influencer]['influencer_main_genre'].values[0]
    age_sati_rank_new[influencer]=(age_sati_rank[age][field][influencer]+age_sati_rank[age+10][field][influencer]+age_sati_rank[age+20][field][influencer])/3

In [524]:
pop60data=age_sati_rank[1970]['Pop/Rock']

In [525]:
pop60data_sort=sorted(pop60data.items(), key = lambda kv:(kv[1], kv[0]))

In [526]:
pop60data_sort[0:15][0]

(754032, 1)

In [527]:
id_list=[]
for j in pop60data_sort[0:15]:
    id_list.append(j[0])

In [528]:
name_list=[]

In [543]:
for ida in id_list:
    name_list.append(data[data['follower_id']==ida]['follower_name'].values[0])

KeyboardInterrupt: 

In [542]:
fac4_df=pd.DataFrame()
fac4_df['name']=name_list
fac4_df.to_csv('fact4.csv')

# factor5 艺术生涯竞争力
艺术生涯竞争力（艺术生涯的所有领域的艺术家总人数）（竞争力越大，影响力越大）

1970、1980、1990三年所在领域的影响力排名均值

In [293]:
age_sati_year={1930:0,1940:0,1950:0,1960:0,1970:0,1980:0,1990:0,2000:0,2010:0,2020:0,2030:0}

In [296]:
for age_num in age_sati:
    for type_num in age_sati[age_num]:
        age_sati_year[age_num]=age_sati_year[age_num]+age_sati[age_num][type_num]
    

In [297]:
age_sati_year#各年总人数

{1930: 155,
 1940: 187,
 1950: 231,
 1960: 288,
 1970: 335,
 1980: 368,
 1990: 401,
 2000: 472,
 2010: 540,
 2020: 515,
 2030: 283}

In [299]:
data.head()

,influencer_id,influencer_name,influencer_main_genre,influencer_active_start,follower_id,follower_name,follower_main_genre,follower_active_start
0,759491,The Exploited,Pop/Rock,1980,74,Special Duties,Pop/Rock,1980
1,25462,Tricky,Electronic,1990,335,PJ Harvey,Pop/Rock,1990
2,66915,Bob Dylan,Pop/Rock,1960,335,PJ Harvey,Pop/Rock,1990
3,71209,Leonard Cohen,Pop/Rock,1950,335,PJ Harvey,Pop/Rock,1990
4,91438,The Gun Club,Pop/Rock,1980,335,PJ Harvey,Pop/Rock,1990


In [306]:
age_sati_year_per_artist={}#储存每个艺术家艺术生涯中的总人数
for influencer in influencer_id:
    active_age=data[data['influencer_id']==influencer]['influencer_active_start'].values[0]
    age_sati_year_per_artist[influencer]=age_sati_year[active_age]+age_sati_year[active_age+10]+age_sati_year[active_age+20]

In [307]:
for follower in follower_id:
    active_age=data[data['follower_id']==follower]['follower_active_start'].values[0]
    age_sati_year_per_artist[follower]=age_sati_year[active_age]+age_sati_year[active_age+10]+age_sati_year[active_age+20]

# factor 6 艺术生涯内追随者数目


In [323]:
now_follower_sati={}#艺术生涯内追随者数目
for follower in influencer_id:
    now_df=data[data['influencer_id']==follower]
    now_df=now_df[now_df['follower_active_start']<now_df['influencer_active_start'].values[0]+20]
    now_follower_sati[follower]=now_df['follower_active_start'].shape[0]


In [327]:
for follower in follower_id:
    now_follower_sati[follower]=0

# factor 7 艺术生涯外追随者数目

In [337]:
out_follower_sati={}#艺术生涯外追随者数目
for influencer in influencer_id:
    now_df=data[data['influencer_id']==influencer]
    if now_df['follower_active_start'].shape[0]>0:
        now_df=now_df[now_df['follower_active_start']>now_df['influencer_active_start'].values[0]+20]
        out_follower_sati[influencer]=now_df['follower_active_start'].shape[0]
    else:
        out_follower_sati[influencer]=0
for follower in follower_id:
    out_follower_sati[follower]=0        

In [334]:
now_df=data[data['influencer_id']==66915]
now_df=now_df[now_df['follower_active_start']>now_df['influencer_active_start'].values[0]+20]
now_df['follower_active_start'].shape[0]

89

In [342]:
out_follower_sati[66915]#检验正确

89

# factor 8 追随者跨的领域数目

In [347]:
data[data['influencer_id']==66915]['follower_main_genre'].value_counts().shape[0]

8

In [348]:
follower_trans_sati={}#追随者跨的领域数目
for influencer in influencer_id:
    #follower_df=data[data['influencer_id']==influencer]['follower_main_genre'].value_counts().shape[0]
    follower_trans_sati[influencer]=data[data['influencer_id']==influencer]['follower_main_genre'].value_counts().shape[0]
for follower in follower_id:
    follower_trans_sati[follower]=0

# 综合

In [350]:
node_list=[]
for follower in follower_id:
    node_list.append(follower)
for influencer in influencer_id:
    node_list.append(influencer)

In [353]:

sati=pd.DataFrame(index=node_list)

In [359]:
distance_line=[]
for node in node_list:
    distance_line.append(distances[node])
sati['distance importance']=distance_line

In [360]:
line=[]
for node in node_list:
    line.append(field_num[node])
sati['field num']=line

In [397]:
line=[]
for node in node_list:
    line.append(field_age_people[node])
sati['in age field num']=line

In [398]:
line=[]
for node in node_list:
    line.append(age_sati_rank_new[node])
sati['in age avg rank']=line

In [399]:
line=[]
for node in node_list:
    line.append(now_follower_sati[node])
sati['in age follower num']=line

In [400]:
line=[]
for node in node_list:
    line.append(out_follower_sati[node])
sati['out age follower num']=line

In [401]:
line=[]
for node in node_list:
    line.append(follower_trans_sati[node])
sati['trans field follower num']=line

In [411]:
line=[]
for node in node_list:
    line.append(age_sati_year_per_artist[node])
sati['in age compete']=line

In [412]:
sati.loc[66915]

1:distance importance          2.332947
field num                   2808.000000
in age field num              53.000000
in age avg rank                2.000000
in age follower num          241.000000
out age follower num          89.000000
trans field follower num       8.000000
in age compete               991.000000
Name: 66915, dtype: float64

In [413]:
sati.to_csv('problem1 factors.csv')

# 相关性分析

In [414]:
sati.corr().to_csv('相关系数.csv')

In [415]:
sati.corr()

,1:distance importance,field num,in age field num,in age avg rank,in age follower num,out age follower num,trans field follower num,in age compete
1:distance importance,1.000000,0.219057,0.475528,-0.167248,-0.276371,-0.401394,-0.546260,0.565118
field num,0.219057,1.000000,0.344213,0.744263,0.096541,-0.030525,-0.181259,0.334991
in age field num,0.475528,0.344213,1.000000,0.104348,-0.155329,-0.178514,-0.321017,0.523852
in age avg rank,-0.167248,0.744263,0.104348,1.000000,0.264111,0.026935,0.008186,0.199769
in age follower num,-0.276371,0.096541,-0.155329,0.264111,1.000000,0.515663,0.480730,-0.140309
out age follower num,-0.401394,-0.030525,-0.178514,0.026935,0.515663,1.000000,0.556198,-0.298574
trans field follower num,-0.546260,-0.181259,-0.321017,0.008186,0.480730,0.556198,1.000000,-0.506553
in age compete,0.565118,0.334991,0.523852,0.199769,-0.140309,-0.298574,-0.506553,1.000000


In [450]:
sati.corr().index[4]

'in age follower num'

In [429]:
corr=sati.corr()

In [457]:
list1=[]
for i in range(0,8):
    list1.append([i,7,corr[sati.corr().index[7]].values.tolist()[i]])

In [458]:
list1

[[0, 7, 0.5651182622803844],
 [1, 7, 0.33499106457186517],
 [2, 7, 0.5238516634042835],
 [3, 7, 0.19976912847588768],
 [4, 7, -0.1403092641460446],
 [5, 7, -0.2985742560367833],
 [6, 7, -0.5065529636116348],
 [7, 7, 1.0]]